In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import datetime
from urllib.parse import urlparse

In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

 
driver.get("https://www.bbc.com/news")
print(driver.title)

Home - BBC News


In [10]:
news_urls=[]
def get_urls():
 elements_for_url=driver.find_elements(By.CLASS_NAME, 'gs-c-promo-heading')
 for element in elements_for_url:
        news_urls.append(element.get_attribute('href'))
 print(len(elements_for_url))
get_urls()
print(len(news_urls))#list with all the urls to the articels. should be equally long to elements_for_url
print(news_urls[64])

79
79
https://www.bbc.com/sport/american-football/67584335


In [14]:
def remove_numbers(category):
    # Find the position of the first digit in the category
    digit_index = next((index for index, char in enumerate(category) if char.isdigit()), None)

    # Remove everything after the first digit
    category_without_number = category[:digit_index] if digit_index is not None else category

    # Remove trailing hyphen
    if category_without_number.endswith('-'):
        category_without_number = category_without_number[:-1]

    return category_without_number

def get_topic(url):
 try:
  parsed_url = urlparse(url)
  path_segments = parsed_url.path.split('/')

# Find the segment after "news"
  category_segment_index = path_segments.index("news") + 1
  category = path_segments[category_segment_index]

  category_without_number = remove_numbers(category)

  return category_without_number
 except:

  try:
   parsed_url = urlparse(url)
   path_segments = parsed_url.path.split('/')
   category_segment_index = path_segments.index("sport") + 1
   category = path_segments[category_segment_index]


   category_without_number = remove_numbers(category)

   return category_without_number
 
  except:
   
   return "no info"


5 hours ago
5 hours ago


In [4]:
#here we extract all the text contents from the articles. This takes approximately 2 minutes with a total of 76 articels today
def extract_contents(text_contents):
  try:
   article = driver.find_element(By.TAG_NAME, 'article')
   text_contents.append(article.text)
  except: #exception if no article element is there. doesn´t happen often, but has to be handled
   article = driver.find_element(By.TAG_NAME, 'body')
   text_contents.append(article.text)
   print("article not found") #in this test, 3 of 73 articels had no article element. 
#extract_contents()
#print(len(text_contents)) 

In [5]:

#here are a few exceptions, because the bbc articles have different html structures
def get_headers(headers):
  try:
   header=driver.find_element(By.ID, 'main-heading')
   headers.append(header.text)
  except:
   try:
     header = driver.find_element(By.CLASS_NAME, 'qa-story-headline')
     headers.append(header.text)
   except:
    try:
     header = driver.find_element(By.CLASS_NAME, 'article-headline__text')
     headers.append(header.text)
    except:
     header=None
     headers.append(header)
     print('no header')
#get_headers()




In [6]:
def get_timestamps(time):
  try:
   time_element=driver.find_element(By.TAG_NAME, 'time')
   date=(time_element.get_attribute('datetime'))
   time.append(date.split(".")[0])#ignore the error, it works when it´s accessed by the main function
  except:
    date=None
    time.append(date)
    print('no date')

In [7]:

def get_Image(imageURL,ImageDesc):
   try:
    image_element=driver.find_element(By.TAG_NAME, 'img')
    url=(image_element.get_attribute('src'))
    desc=(image_element.get_attribute('alt'))
    imageURL.append(url)
    ImageDesc.append(desc)
   except:
    url=None
    desc=None
    imageURL.append(url)
    ImageDesc.append(desc)
    print('no image')

In [8]:
#main method to extract all contents except the authors(might follow later). Runs way faster than the different smaller methods before
text_contents=[]
time=[]
headers=[]
imageURL=[]
imageDesc=[]
topic=[]
def extract_all():
    for i in range(len(news_urls)):
       driver.get(news_urls[i]) #we go to the specific article by using the url
       parsed_url = urlparse(news_urls[i])
       extract_contents(text_contents)
       get_timestamps(time)
       get_headers(headers)
       get_Image(imageURL,imageDesc)
       topic_result = get_topic(news_urls[i])
       topic.append(topic_result)
extract_all()
print(text_contents[10])
print(time[10])
print(headers[10])
print(imageURL[10])
print(imageDesc[10])
print(topic[10])


no date
no header
article not found
no date
article not found
no date
no header
no date
no date
no date
no date
no date
no date
no date
no date
no header
Elon Musk launches profane attack on X advertisers
Published
11 hours ago
Share
IMAGE SOURCE,
GETTY IMAGES
By James Clayton
BBC North America technology reporter
In a profanity-laced outburst, Elon Musk has slammed advertisers that have left X, warning they will kill the social media platform.
At an event in New York, he accused companies that have joined an ad boycott of the site formerly known as Twitter of trying to blackmail him.
"Go [expletive] yourself," the billionaire said in an interview.
Some firms have paused advertising on X amid concerns over antisemitism, including a post from Mr Musk himself.
The Tesla and SpaceX boss apologised on Wednesday for that post, saying it might be the "dumbest" thing he has ever shared online.
But it was his response to a question about an advertising boycott by companies including Disney, Ap

In [9]:
for item in headers:
  print(item)
print(len(headers))
counter=0
for i in range(len(headers)):
    if headers[i]==None:
      counter=counter+1
print(counter)
for item in news_urls:
  print(item)

None
Bedouin family desperate for news on teenagers held hostage in Gaza
Nearly 100,000 Gaza buildings may be damaged, satellite images show
COP28: Poor countries win 30-year fight for climate cash
Shane MacGowan: Pogues singer dies aged 65
Shannen Doherty reveals cancer has spread to her bones
Russian court bans 'LGBT movement'
Former US Secretary of State Henry Kissinger dies aged 100
Kayaker's 20m drop down ice waterfall in Svalbard, Norway
Ukraine: Russian general 'blown up on mine'
Elon Musk launches profane attack on X advertisers
Ukraine: Russian general 'blown up on mine'
Elon Musk launches profane attack on X advertisers
Swedish murder: Five teenagers found guilty over taxi driver's killing
Inactive Google accounts to be deleted from Friday
OpenAI chaos not about AI safety, says Microsoft boss
Israel-Hamas truce in Gaza extended as more hostages freed
Three Israelis killed by Palestinian gunmen at Jerusalem bus stop
Hamas hostages: Relief as Thai hostage is reunited with famil

In [10]:
def get_authors(article):
    authors = []
    lines = article.split('\n')
    for line in lines[:13]:  # Loop through lines from lines[0] to lines[10]
        if "By" in line:
            authors.append(line.replace('By', ''))
    return authors

all_authors = []
def get_all_authors():
    for article in text_contents:
        authors = get_authors(article)
        if not authors:  # Check if authors is an empty list
            all_authors.append(None)
        else:
            all_authors.extend(authors)

# Print the authors
for author in all_authors:
    print(author)


get_all_authors()
print(all_authors[10])
print(len(all_authors))
print(all_authors)

counter=0
for i in range(len(all_authors)):
    if all_authors[i]==None:
      counter=counter+1
print(counter)

#<p class="lx-commentary__meta-reporter gel-long-primer" data-reactid=".18n36quudgy.0.2.0.0.1.0.1.0.$lx-tabs0.0.$lx-commentary.$lx-commentary.0.2.0.0">  Edited by Nadia Ragozhina and Samuel Horti</p>
#<div class="ssrcss-68pt20-Text-TextContributorName e8mq1e96">By Jeremy Bowen</div>
#<div class="ssrcss-68pt20-Text-TextContributorName e8mq1e96">By David Gritten</div>

 James Clayton
76
[None, ' Anna Foster', ' Dominic Bailey, Erwan Rivault & Daniele Palumbo', ' Matt McGrath', ' Steven McIntosh, Helen Bushby and Annabel Rackham', ' Annabel Rackham', ' Steve Rosenberg', ' Bernd Debusmann Jr', None, ' Robert Greenall', ' James Clayton', ' Robert Greenall', ' James Clayton', ' Ido Vock', ' Shiona McCallum', ' Faisal Islam & Shiona McCallum', ' Ece Goksedef', ' David Gritten', None, ' Wyre Davies', ' Sam Hancock & Lou Newton', None, None, None, ' Hannah Price', ' Mark Poynting', None, ' Frances Mao', ' Anthony Zurcher', None, None, None, None, None, ' Steven Brocklehurst', None, ' MaryLou Costa', ' Thomas Mackintosh', ' Joel Guinto', ' Riyah Collins', None, ' Steven McIntosh, Helen Bushby and Annabel Rackham', ' Annabel Rackham', ' Anna Foster', ' Robert Greenall', ' James Clayton', ' Dominic Bailey, Erwan Rivault & Daniele Palumbo', ' Ido Vock', ' Danai Nesta Kupemba', ' Steve Rosenberg', ' Tom Gerken', ' Martha Henriques', ' Sam Becker', ' Carolina Dra

In [21]:
#dictionary with all infos, ordered after attributes
from datetime import datetime
import pytz

germany_timezone = pytz.timezone("Europe/Berlin")


articles_info={}
for i in range(len(news_urls)):
 article_info_i={ 
  "headline": headers[i],
  "url": news_urls[i],
  "date":time[i],
  "authors":[all_authors[i]],
  "ImageURL":imageURL[i],
  "ImageDescription":imageDesc[i],
  "scrapingTimeStamp":(datetime.now(germany_timezone).isoformat()).split(".")[0],
  "source":"bbc",
  "topics":topic[i],
  "contents":text_contents[i]
 }
 articles_info[i]=article_info_i
print(articles_info[50])
print(topic[65])
#URLId, Headline, Contents, Authors, UploadDate, ReadTime, ImageURL, ImageDescription



{'headline': 'Palestinian activist Ahed Tamimi freed by Israel', 'url': 'https://www.bbc.com/news/world-middle-east-67574546', 'date': '2023-11-30T13:50:56', 'authors': [' Sam Hancock & Lou Newton'], 'ImageURL': 'https://ichef.bbci.co.uk/news/976/cpsprodpb/837A/production/_131885633_tamimi.jpg', 'ImageDescription': 'Palestinian activist Ahed Tamimi is pictured with her mother after being returned to the West Bank', 'scrapingTimeStamp': '2023-11-30T19:12:24', 'source': 'bbc', 'contents': 'Palestinian activist Ahed Tamimi freed by Israel\nPublished\n4 hours ago\nShare\nRelated Topics\nIsrael-Gaza war\nIMAGE SOURCE,\nREUTERS\nImage caption,\nAhed Tamimi, 22, was seen reuniting with her mother in Ramallah in the West Bank\nBy Sam Hancock & Lou Newton\nBBC News\nActivist Ahed Tamimi was one of 30 Palestinian prisoners released by Israel on Wednesday, as part of the truce deal with Hamas.\nIsrael\'s prison service said a mixture of men and women had been freed "from a number of [its] facilit

In [19]:
import psycopg2

In [16]:
conn = psycopg2.connect(dbname="postgres",user="postgres", password="postgres", port="5432", host="localhost")
conn.autocommit = True
cursor = conn.cursor()

In [23]:
for article in articles_info.values():
    try:
        cursor.execute('''INSERT INTO Articles(URLId, Headline, Content, Authors, UploadTimestamp, ImageURL, ImageDescription,scrapingTimeStamp, source, topic) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);''', 
                       (article["url"], article["headline"], article["contents"], article["authors"], article["date"], article["ImageURL"], article["ImageDescription"], article["scrapingTimeStamp"], article["source"], article["topics"])
                       )
    except psycopg2.IntegrityError as e:
        if "duplicate key value violates unique constraint" in str(e):
            print(f"Article with URLId {article['url']} already exists in the database.")






Article with URLId https://www.bbc.com/news/world-europe-67579357 already exists in the database.
Article with URLId https://www.bbc.com/news/world-us-canada-67574396 already exists in the database.
Article with URLId https://www.bbc.com/news/world-67578559 already exists in the database.
Article with URLId https://www.bbc.com/news/world-us-canada-67574846 already exists in the database.
Article with URLId https://www.bbc.com/news/entertainment-arts-67546785 already exists in the database.
Article with URLId https://www.bbc.com/news/entertainment-arts-67577983 already exists in the database.
Article with URLId https://www.bbc.com/news/world-middle-east-67573069 already exists in the database.
Article with URLId https://www.bbc.com/news/world-europe-67579357 already exists in the database.
Article with URLId https://www.bbc.com/news/world-us-canada-67574396 already exists in the database.
Article with URLId https://www.bbc.com/news/world-middle-east-67565872 already exists in the databa